In [4]:
import pandas as pd
from datetime import datetime, timedelta
import ast

file_path = "bills.csv"

daily sales

In [5]:
def process_daily_sales(file_path):
   
    try:
        bills_df = pd.read_csv(file_path)
        bills_df['Date_Time'] = pd.to_datetime(bills_df['Date_Time'])
    except FileNotFoundError:
        print("Error: The file was not found.")
        return {}

    daily_sales_data = {}
    start_date = datetime(2022, 6, 1)
    end_date = datetime(2025, 5, 31)

    current_date = start_date
    while current_date <= end_date:
        bills_for_today = bills_df[bills_df['Date_Time'].dt.date == current_date.date()]
        if not bills_for_today.empty:
            todays_items = []
            for _, bill in bills_for_today.iterrows():
                try:
                    product_ids = ast.literal_eval(bill['Product_IDs'])
                    quantities = ast.literal_eval(bill['Quantities'])
                    for i in range(len(product_ids)):
                        todays_items.append({
                            'product_id': product_ids[i],
                            'quantity_sold': quantities[i]
                        })
                except (ValueError, SyntaxError):
                    continue
            daily_sales_data[current_date.strftime('%Y-%m-%d')] = todays_items
        else:
            daily_sales_data[current_date.strftime('%Y-%m-%d')] = []
        current_date += timedelta(days=1)

    return daily_sales_data  



daily_data = process_daily_sales(file_path)
all_sales_daily = []
for date, sales in daily_data.items():
    for item in sales:
        all_sales_daily.append({
            "Date": date,
            "ProductID": item['product_id'],
            "Quantity_Sold": item['quantity_sold']
        })

daily_sales_df = pd.DataFrame(all_sales_daily)
if not daily_sales_df.empty:
    summary_daily_df = (
        daily_sales_df.groupby(["Date", "ProductID"], as_index=False)["Quantity_Sold"]
        .sum()
        .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
    )
    summary_daily_df.to_csv("daily_sales_summary.csv", index=False)
    


  

weekly sales

In [6]:
def process_weekly_sales(file_path):
    
    try:
        bills_df = pd.read_csv(file_path)
        bills_df['Date_Time'] = pd.to_datetime(bills_df['Date_Time'])
    except FileNotFoundError:
        print(f"Error: The file was not found.")
        return {}

    weekly_sales_data = {}
    start_date = datetime(2022, 6, 1)
    end_date = datetime(2025, 5, 31)

    
    start_date = start_date - timedelta(days=start_date.weekday())
    current_date = start_date

    while current_date <= end_date:
        week_start = current_date
        week_end = current_date + timedelta(days=6)
        bills_for_week = bills_df[
            (bills_df['Date_Time'].dt.date >= week_start.date()) &
            (bills_df['Date_Time'].dt.date <= week_end.date())
        ]
        if not bills_for_week.empty:
            weeks_items = []
            for _, bill in bills_for_week.iterrows():
                try:
                    product_ids = ast.literal_eval(bill['Product_IDs'])
                    quantities = ast.literal_eval(bill['Quantities'])
                    for i in range(len(product_ids)):
                        weeks_items.append({
                            'product_id': product_ids[i],
                            'quantity_sold': quantities[i]
                        })
                except (ValueError, SyntaxError):
                    continue
            weekly_sales_data[week_start.strftime('%Y-%m-%d')] = weeks_items
        else:
            weekly_sales_data[week_start.strftime('%Y-%m-%d')] = []
        current_date += timedelta(days=7)

    return weekly_sales_data 


weekly_data = process_weekly_sales(file_path)
all_sales_weekly = []
for week_start, sales in weekly_data.items():
    for item in sales:
        all_sales_weekly.append({
            "Week_Start": week_start,
            "ProductID": item['product_id'],
            "Quantity_Sold": item['quantity_sold']
        })

weekly_sales_df = pd.DataFrame(all_sales_weekly)
if not weekly_sales_df.empty:
    summary_weekly_df = (
        weekly_sales_df.groupby(["Week_Start", "ProductID"], as_index=False)["Quantity_Sold"]
        .sum()
        .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
    )
    summary_weekly_df.to_csv("weekly_sales_summary.csv", index=False)

   

monthly sales

In [7]:

def process_monthly_sales(file_path):
    try:
        bills_df = pd.read_csv(file_path)
        bills_df['Date_Time'] = pd.to_datetime(bills_df['Date_Time'])
    except FileNotFoundError:
        return {}

    monthly_sales_data = {}
    start_date = datetime(2022, 6, 1)
    end_date = datetime(2025, 5, 31)

    current_date = start_date.replace(day=1)
    while current_date <= end_date:
        
        if current_date.month == 12:
            next_month = datetime(current_date.year + 1, 1, 1)
        else:
            next_month = datetime(current_date.year, current_date.month + 1, 1)

        bills_for_month = bills_df[
            (bills_df['Date_Time'].dt.date >= current_date.date()) &
            (bills_df['Date_Time'].dt.date < next_month.date())
        ]
        if not bills_for_month.empty:
            month_items = []
            for _, bill in bills_for_month.iterrows():
                try:
                    product_ids = ast.literal_eval(bill['Product_IDs'])
                    quantities = ast.literal_eval(bill['Quantities'])
                    for i in range(len(product_ids)):
                        month_items.append({
                            'product_id': product_ids[i],
                            'quantity_sold': quantities[i]
                        })
                except (ValueError, SyntaxError):
                    continue
            monthly_sales_data[current_date.strftime('%Y-%m')] = month_items
        else:
            monthly_sales_data[current_date.strftime('%Y-%m')] = []
        current_date = next_month

    return monthly_sales_data



monthly_data = process_monthly_sales(file_path)
all_sales_monthly = []
for month, sales in monthly_data.items():
    for item in sales:
        all_sales_monthly.append({
            "Month": month,
            "ProductID": item['product_id'],
            "Quantity_Sold": item['quantity_sold']
        })

monthly_sales_df = pd.DataFrame(all_sales_monthly)
if not monthly_sales_df.empty:
    summary_monthly_df = (
        monthly_sales_df.groupby(["Month", "ProductID"], as_index=False)["Quantity_Sold"]
        .sum()
        .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
    )
    summary_monthly_df.to_csv("monthly_sales_summary.csv", index=False)
    

In [1]:
import pandas as pd

# Load CSV
df = pd.read_csv("monthly_sales_summary.csv")

# Group by Product_ID and sum all sales
total_sales = df.groupby("ProductID")["Total_Quantity_Sold"].sum().reset_index()

# Rename for clarity
total_sales.columns = ["Product_ID", "Total_Sold"]

# Save to CSV
total_sales.to_csv("totalmonthly_sales_summary.csv", index=False)

print("Total sales per Product_ID saved to 'totalweekly_sales_summary.csv'")


Total sales per Product_ID saved to 'totalweekly_sales_summary.csv'
